In [ ]:
# !pip install geemap
# !pip install geopandas
# !pip install geojson

In [8]:
import os
import sys
import ee

In [85]:
# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=yIrlGSn_W-yxPNKcHPccq74sRM86HwQeQ0uGaP2zvcQ&tc=yr9MO6tU2UWZCwhMf1gv1-Wx_CbFgo5ezDCz_F6g0VE&cc=QkWhuw2taNWnh26mLjG0eKuh96k4AVQWedrqNqEBTtA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvs3a2cg7_Hp76ffp1rGOX_h5Hmps2-FBu8XL4g4LbC7NPwyQT71TN4

Successfully saved authorization token.


In [86]:
# Initialize the library.
ee.Initialize()

In [11]:
import geemap

In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# nb_path = '/content/colab_packages'
# os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
# sys.path.insert(0, nb_path)  # or append(nb_path)
# #The last three lines are what changes the path of the file.

In [ ]:
# !pip install --target=$nb_path geemap

In [ ]:
# directory = '/content/drive/MyDrive/lst_files/aoi/ctg-polygon.shp'

In [13]:
Map = geemap.Map()
aoi_shp =  '/content/drive/MyDrive/lst_files/kutupalong/Kutupalong-AOI-Polygon-polygon.shp'
aoi = geemap.shp_to_ee(aoi_shp)
Map.addLayer(aoi, {}, 'AOI')
Map.centerObject(aoi, zoom=16)
Map

Map(center=[21.198233064130157, 92.15051686742379], controls=(WidgetControl(options=['position', 'transparent_…

In [14]:
image_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
image_collection = image_collection.filterMetadata('WRS_PATH','equals',136).filterMetadata('WRS_ROW', 'equals',45).filterDate('2021-01-01', '2021-12-31').sort('CLOUD_COVER')
# image_collection = image_collection.filterMetadata('WRS_PATH','equals',126).filterMetadata('WRS_ROW', 'equals',52).filterDate('2021-01-01', '2021-12-31').sort('CLOUD_COVER')
print("Total Image Downloaded: ",image_collection.size().getInfo()) #gets the total number of images


Total Image Downloaded:  16


In [15]:
least_cloudy_image = image_collection.first()

In [17]:
least_cloudy_image.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7801],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 241485, 0, -30, 2513715]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7801],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 241485, 0, -30, 2513715]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7801],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 241485, 0, -30, 2513715]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7801],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 241485, 0, -30, 2513715]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7801],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 241485, 0, -30, 2513715]},
  {'id': 'B6',
   'data_type': {'type': 'Pi

In [18]:
Map.addLayer(least_cloudy_image)
Map.centerObject(least_cloudy_image)
Map

Map(bottom=7377657.0, center=[21.198233064130157, 92.15051686742379], controls=(WidgetControl(options=['positi…

In [19]:
# Define the visualization parameters.

image_viz_params = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1]
}
Map.addLayer(least_cloudy_image, image_viz_params, "Least Cloudy Image")
Map.centerObject(least_cloudy_image)
Map

Map(bottom=14663.0, center=[21.85926828402503, 91.57561698270463], controls=(WidgetControl(options=['position'…

In [20]:
#Picking the required bands
nir_band = least_cloudy_image.select('B5');
red_band = least_cloudy_image.select('B4');
thermal_band = least_cloudy_image.select('B10');

In [21]:
#cliping the band specific images for the AOI
nir_band_clipped = nir_band.clip(aoi)
red_band_clipped = red_band.clip(aoi)
thermal_band_clipped = thermal_band.clip(aoi);

Map.addLayer(nir_band_clipped,{},'clipped_NIR')
Map.centerObject(aoi)
Map.addLayerControl()
Map

Map(bottom=14618.0, center=[22.12381042563998, 91.6267995528926], controls=(WidgetControl(options=['position',…

In [78]:
# Compute the NDVI for clipped images/aoi
ndvi = (nir_band_clipped.subtract(red_band_clipped)).divide(  (nir_band_clipped.add(red_band_clipped)) ).rename('NDVI')

# visualize the ndvi
ndviPar = {
    'min': 0.4, 
    'max': 0.54, 
    'palette': ['red', 'yellow', 'green']
           }

Map.addLayer(ndvi, ndviPar, 'NDVI of AOI')
Map.centerObject(ndvi, zoom = 11)
Map


Map(bottom=752.0, center=[21.667712612943202, 91.61241607266717], controls=(WidgetControl(options=['position',…

In [23]:
ndvi.getInfo()

{'type': 'Image',
 'bands': [{'id': 'NDVI',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7801],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 241485, 0, -30, 2513715]}]}

In [25]:
##MIN NDVI VALUE

min_ndvi = ee.Number(ndvi.reduceRegion(**{
  "reducer": ee.Reducer.min(),
  "scale": 30,
  "maxPixels": 10**9
  }).values().get(0))
print('min value of NDVI',min_ndvi.getInfo())

min value of NDVI -0.37243403175430867


In [27]:
max_ndvi = ee.Number(ndvi.reduceRegion(**{
"reducer": ee.Reducer.max(),
"scale": 30,
"maxPixels": 10**9
}).values().get(0));
print('max value of NDVI',max_ndvi.getInfo())

max value of NDVI 0.738098396337262


In [28]:
#proportional vegetation

pv =(ndvi.subtract(min_ndvi).divide(max_ndvi.subtract(min_ndvi))).pow(ee.Number(2)).rename('PV')

In [30]:
pv_max = ee.Number(pv.reduceRegion(**{
  "reducer": ee.Reducer.max(),
  "scale": 30,
  "maxPixels": 10**9
}).values().get(0));
print( 'max value of pv',pv_max.getInfo())

max value of pv 1


In [31]:
##MIN pv VALUE

min_pv = ee.Number(pv.reduceRegion(**{
  "reducer": ee.Reducer.min(),
  "scale": 30,
  "maxPixels": 10**9
  }).values().get(0))
print('min value of pv',min_pv.getInfo())




min value of pv 0


In [45]:
Map.addLayer(pv, ndviPar, 'Proportional Vegetation')
Map.centerObject(pv, zoom = 11)
Map.clear_layers()
Map.add_basemap()
Map.center_object(aoi)
Map.add_layer(pv)
Map

Map(bottom=14742.0, center=[21.667712612943202, 91.61241607266717], controls=(WidgetControl(options=['position…

In [57]:
#Emissivity
a= ee.Number(0.004);
b= ee.Number(0.986);
em=pv.multiply(a).add(b).rename('EM');

# Map.addLayer(em, {min:0.98, max:0.99, palette: ['gray','yellow','green']}, 'Em');

In [54]:
# minmax = ee.Number(pv.reduceRegion(**{
#   "reducer": ee.Reducer.minMax(),
#   "scale": 30,
#   "maxPixels": 10**9
#   }).values())


In [58]:
# minmax.getInfo()

In [59]:
# minmax.getInfo()

In [60]:
emmax = ee.Number(em.reduceRegion(**{
  "reducer": ee.Reducer.max(),
  "scale": 30,
  "maxPixels": 10**9
  }).values().get(0))
print(emmax.getInfo())

0.99


In [61]:
emmin = ee.Number(em.reduceRegion(**{
  "reducer": ee.Reducer.min(),
  "scale": 30,
  "maxPixels": 10**9
  }).values().get(0))
print(emmin.getInfo())

0.986


In [62]:
# Land surface temperature (LST) in C
LST = thermal_band_clipped.expression(
'(Tb/(1 + (10.8* (Tb / 14388))*log(EM)))-273.15', {
 'Tb': thermal_band_clipped,
'EM': em.select('EM')
}).rename('LST')


In [63]:
LSTParams = {
    'min':27.115292564360516, 
    'max':31.653865373204212, 
    'palette': ['gray','yellow','green']
    }

In [64]:
lstmin = ee.Number(LST.reduceRegion(**{
  "reducer": ee.Reducer.min(),
  "scale": 30,
  "maxPixels": 10**9
  }).values().get(0))
print(lstmin.getInfo())

24.61047603870543


In [65]:
lstmax = ee.Number(LST.reduceRegion(**{
  "reducer": ee.Reducer.max(),
  "scale": 30,
  "maxPixels": 10**9
  }).values().get(0))
print(lstmax.getInfo())

33.958986269391005


In [68]:
Map.addLayer(LST, LSTParams, 'LandSurfaceTemperature')
Map.center_object(aoi,zoom=5)
Map

Map(bottom=2101.0, center=[21.198233064130157, 92.15051686742379], controls=(WidgetControl(options=['position'…

In [88]:
# // Export the image, specifying the CRS, transform, and region.
task = ee.batch.Export.image.toDrive(image=LST,
                                     scale=30,
                                     fileFormat='GeoTIFF',
                                     description='Kutupalong',
                                     folder='lst_files',
                                     maxPixels=1e9)

task.start()